# COVID-19 Time Series: Анализ волн заболеваемости

Разбираемся с волнами COVID - когда были пики, как быстро она распространялась

Используем данные из ноутбука 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# загружаем данные
df = pd.read_csv('data/covid_clean.csv')
df['Date'] = pd.to_datetime(df['Date'])

print(f'Данные загружены: {df.shape[0]} строк')
print(df.head())

## Разложение временного ряда (Time Series Decomposition)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# берем Россию для примера
russia_data = df[df['Country'] == 'Russia'].sort_values('Date')
russia_data = russia_data.set_index('Date')['Confirmed']

# разложение временного ряда
# помогает понять тренд, сезонность и шум
decomposition = seasonal_decompose(russia_data, model='additive', period=365)

fig, axes = plt.subplots(4, 1, figsize=(14, 10))

# исходные данные
axes[0].plot(russia_data.index, russia_data.values, color='steelblue', linewidth=1.5)
axes[0].set_ylabel('Случаи')
axes[0].set_title('COVID-19 в России: Исходные данные')
axes[0].grid(True, alpha=0.3)

# тренд (долгосрочное направление)
axes[1].plot(decomposition.trend.index, decomposition.trend.values, color='red', linewidth=2)
axes[1].set_ylabel('Тренд')
axes[1].set_title('Долгосрочный тренд')
axes[1].grid(True, alpha=0.3)

# сезонность (повторяющиеся паттерны)
axes[2].plot(decomposition.seasonal.index, decomposition.seasonal.values, color='green', linewidth=1)
axes[2].set_ylabel('Сезонность')
axes[2].set_title('Сезонные паттерны')
axes[2].grid(True, alpha=0.3)

# остаток (шум и аномалии)
axes[3].plot(decomposition.resid.index, decomposition.resid.values, color='purple', linewidth=0.8)
axes[3].set_ylabel('Остаток')
axes[3].set_xlabel('Дата')
axes[3].set_title('Остаток (шум + аномалии)')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/05_time_series_decomposition.png', dpi=300, bbox_inches='tight')
plt.show()

print('Разложение выполнено!')

## Скользящее среднее и обнаружение волн

In [ ]:
# считаем 7-дневное скользящее среднее (сглаживаем шум)
russia_data_df = df[df['Country'] == 'Russia'].sort_values('Date').copy()
russia_data_df['MA7'] = russia_data_df['Confirmed'].rolling(window=7, center=False).mean()
russia_data_df['MA30'] = russia_data_df['Confirmed'].rolling(window=30, center=False).mean()

# визуализируем
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(russia_data_df['Date'], russia_data_df['Confirmed'], alpha=0.3, label='Ежедневно', color='steelblue')
ax.plot(russia_data_df['Date'], russia_data_df['MA7'], label='7-день скользящее среднее', linewidth=2, color='orange')
ax.plot(russia_data_df['Date'], russia_data_df['MA30'], label='30-день скользящее среднее', linewidth=2, color='red')

ax.set_xlabel('Дата')
ax.set_ylabel('Количество случаев')
ax.set_title('COVID-19 в России: Скользящие средние (выявление волн)')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/06_moving_average.png', dpi=300, bbox_inches='tight')
plt.show()

## Обнаружение пиков волн

In [ ]:
from scipy.signal import find_peaks

# находим пики волн для разных стран
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

countries_to_plot = ['Russia', 'USA', 'India', 'Germany']

for idx, country in enumerate(countries_to_plot):
    country_data = df[df['Country'] == country].sort_values('Date').copy()
    country_data['MA7'] = country_data['Confirmed'].rolling(window=7).mean()
    
    # находим пики
    peaks, properties = find_peaks(country_data['Confirmed'].values, distance=50, prominence=100)
    
    ax = axes[idx]
    ax.plot(country_data['Date'], country_data['Confirmed'], alpha=0.4, label='Ежедневно')
    ax.plot(country_data['Date'], country_data['MA7'], linewidth=2, label='7-день сред.', color='orange')
    
    # отмечаем пики
    if len(peaks) > 0:
        peak_dates = country_data['Date'].iloc[peaks]
        peak_values = country_data['Confirmed'].iloc[peaks]
        ax.scatter(peak_dates, peak_values, color='red', s=100, marker='v', label='Пики волн', zorder=5)
    
    ax.set_title(f'{country}')
    ax.set_ylabel('Случаи')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/07_peak_detection.png', dpi=300, bbox_inches='tight')
plt.show()

print('Пики обнаружены!')

## Скорость роста случаев (производная)

In [ ]:
# производная показывает скорость изменения
# если положительная - волна растёт, если отрицательная - спадает

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

russia_data_df['Daily_Change'] = russia_data_df['Confirmed'].diff()
russia_data_df['Growth_Rate'] = (russia_data_df['Daily_Change'] / russia_data_df['Confirmed'].shift(1) * 100).fillna(0)

# ежедневные изменения
axes[0].bar(russia_data_df['Date'], russia_data_df['Daily_Change'], 
             color=['green' if x < 0 else 'red' for x in russia_data_df['Daily_Change']], alpha=0.6)
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0].set_ylabel('Ежедневное изменение')
axes[0].set_title('Темп прироста новых случаев')
axes[0].grid(True, alpha=0.3, axis='y')

# процент роста
axes[1].plot(russia_data_df['Date'], russia_data_df['Growth_Rate'], linewidth=1.5, color='steelblue')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].fill_between(russia_data_df['Date'], russia_data_df['Growth_Rate'], 0, alpha=0.3)
axes[1].set_xlabel('Дата')
axes[1].set_ylabel('% изменения')
axes[1].set_title('Процент ежедневного роста/спада')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/08_growth_rate.png', dpi=300, bbox_inches='tight')
plt.show()

## Сравнение волн между странами

In [ ]:
# нормализуем данные чтобы сравнивать волны
# (шкала зависит от размера страны, так что нормализуем)

from sklearn.preprocessing import MinMaxScaler

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

countries_to_compare = ['Russia', 'USA', 'India', 'Brazil']

for idx, country in enumerate(countries_to_compare):
    country_data = df[df['Country'] == country].sort_values('Date').copy()
    
    # нормализуем для сравнения
    scaler = MinMaxScaler()
    normalized = scaler.fit_transform(country_data[['Confirmed']])
    country_data['Normalized'] = normalized
    
    ax = axes[idx]
    ax.plot(country_data['Date'], country_data['Normalized'], linewidth=2, color='steelblue')
    ax.fill_between(country_data['Date'], country_data['Normalized'], alpha=0.3)
    ax.set_title(f'{country} (нормализовано 0-1)')
    ax.set_ylabel('Нормализованные случаи')
    ax.set_ylim([0, 1])
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/09_normalized_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print('Сравнение волн выполнено!')

## Статистический анализ волн

In [ ]:
# анализируем когда были пики для каждой страны
wave_analysis = []

for country in df['Country'].unique():
    country_data = df[df['Country'] == country].sort_values('Date')
    
    # находим максимум
    max_cases = country_data['Confirmed'].max()
    max_date = country_data[country_data['Confirmed'] == max_cases]['Date'].values[0]
    
    max_deaths = country_data['Deaths'].max()
    
    avg_cases = country_data['Confirmed'].mean()
    
    wave_analysis.append({
        'Country': country,
        'Max_Cases': max_cases,
        'Peak_Date': max_date,
        'Max_Deaths': max_deaths,
        'Avg_Daily_Cases': avg_cases
    })

wave_df = pd.DataFrame(wave_analysis).sort_values('Max_Cases', ascending=False)
print('\nАнализ волн по странам:')
print(wave_df)

# сохраняем для дальнейшего анализа
wave_df.to_csv('results/wave_analysis.csv', index=False)
print('\nАнализ сохранён в results/wave_analysis.csv')

In [ ]:
# график: когда были пики
fig, ax = plt.subplots(figsize=(12, 6))

# конвертируем дату в числовой формат для графика
wave_df['Peak_Date_num'] = pd.to_datetime(wave_df['Peak_Date'])
wave_df = wave_df.sort_values('Peak_Date_num')

colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(wave_df)))

ax.scatter(wave_df['Peak_Date_num'], wave_df['Max_Cases'], s=300, c=colors, alpha=0.7, edgecolors='black')

for idx, row in wave_df.iterrows():
    ax.annotate(row['Country'], 
                xy=(row['Peak_Date_num'], row['Max_Cases']),
                xytext=(10, 10), textcoords='offset points',
                fontsize=9, alpha=0.7)

ax.set_xlabel('Дата пика')
ax.set_ylabel('Максимум случаев')
ax.set_title('Когда была пиковая волна COVID в каждой стране')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/10_peaks_timeline.png', dpi=300, bbox_inches='tight')
plt.show()